In [1]:
from lines_generator import dataGenerator
import numpy as np
import keras
from keras import Sequential
from keras.layers import Dense,Conv2D, MaxPooling2D, Flatten

Using TensorFlow backend.


In [2]:
patche_size = 17
input_dim = patche_size**2
num_classes = 2

## ANN Train
Generator

In [3]:
gen_train = dataGenerator(100,'train',prob_peaks=0.2,prob_near_peaks=0.8)
gen_test = dataGenerator(20,'test',prob_peaks=0.2,prob_near_peaks=0.8)

In [10]:
np.load("data/train/im_22.npy").shape

(2087, 3017)

__activations:__ = relu - softmax - sigmoid

In [14]:
# create model
model = Sequential()
model.add(Dense(350, input_dim=input_dim, activation='sigmoid'))
model.add(Dense(250, input_dim=input_dim, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

https://keras.io/models/model/ $\ \ \ $ https://keras.io/optimizers/ $\ \ \ $ https://keras.io/losses/  <br>
__optimizer:__ SGD - RMSprop - Adagrad - Adadelta - Adam - Adamax - Nadam - TFOptimizer. <br>
__loss:__ mean_squared_error - mean_absolute_error - mean_absolute_percentage_error - mean_squared_logarithmic_error - squared_hinge - hinge - categorical_hinge - logcosh - categorical_crossentropy - sparse_categorical_crossentropy - binary_crossentropy - kullback_leibler_divergence - poisson - cosine_proximity.

In [15]:
model.compile(optimizer='Adamax',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
model.fit_generator(gen_train, samples_per_epoch=1024, nb_epoch=50)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1024, epochs=50)`
  """Entry point for launching an IPython kernel.


Epoch 1/50
1024/1024 [==============================] - 41s 40ms/step - loss: 0.2167 - acc: 0.8982
Epoch 2/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1681 - acc: 0.9282
Epoch 3/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1536 - acc: 0.9349
Epoch 4/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1573 - acc: 0.9334
Epoch 5/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1563 - acc: 0.9334
Epoch 6/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1538 - acc: 0.9334
Epoch 7/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1598 - acc: 0.9295
Epoch 8/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1501 - acc: 0.9357
Epoch 9/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1519 - acc: 0.9345
Epoch 10/50
1024/1024 [==============================] - 5s 5ms/step - loss: 0.1545 - acc: 0.9338
Epoch 11/50
1024/1024 [====

In [17]:
score = model.evaluate_generator(gen_test,steps=1024)
score

[0.14428064215161385, 0.9423828125]

In [18]:
pred_test = model.predict_generator(gen_test,steps=128,verbose=1)

128/128 [==============================] - 0s 2ms/step


In [19]:
pred_test

array([[1.3562868e-03, 9.9864370e-01],
       [6.4597488e-04, 9.9935406e-01],
       [1.8472691e-06, 9.9999821e-01],
       ...,
       [9.0339354e-06, 9.9999094e-01],
       [4.3996611e-06, 9.9999559e-01],
       [4.7003723e-06, 9.9999535e-01]], dtype=float32)

## Save

In [20]:
model.save('models/model4.h5')

## CNN Train 
Generator

In [6]:
gen_train = dataGenerator(100,'train',prob_peaks=0.2,prob_near_peaks=0.8,reshape=False,expand_dims=True)
gen_test = dataGenerator(20,'test',prob_peaks=0.2,prob_near_peaks=0.8,reshape=False,expand_dims=True)

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(patche_size,patche_size,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='Adamax',#keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [7]:
model.fit_generator(gen_train, samples_per_epoch=1024, nb_epoch=10)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1024, epochs=10)`
  """Entry point for launching an IPython kernel.


Epoch 1/10
1024/1024 [==============================] - 31s 30ms/step - loss: 0.1765 - acc: 0.9223
Epoch 2/10
1024/1024 [==============================] - 7s 7ms/step - loss: 0.1128 - acc: 0.9545
Epoch 3/10
1024/1024 [==============================] - 7s 7ms/step - loss: 0.0851 - acc: 0.9693
Epoch 4/10
1024/1024 [==============================] - 7s 7ms/step - loss: 0.0813 - acc: 0.9714
Epoch 5/10
1024/1024 [==============================] - 8s 7ms/step - loss: 0.0701 - acc: 0.9752
Epoch 6/10
1024/1024 [==============================] - 8s 7ms/step - loss: 0.0667 - acc: 0.9763
Epoch 7/10
1024/1024 [==============================] - 8s 8ms/step - loss: 0.0549 - acc: 0.9821
Epoch 8/10
1024/1024 [==============================] - 9s 9ms/step - loss: 0.0538 - acc: 0.9813
Epoch 9/10
1024/1024 [==============================] - 8s 8ms/step - loss: 0.0536 - acc: 0.9823
Epoch 10/10
1024/1024 [==============================] - 8s 8ms/step - loss: 0.0473 - acc: 0.9847


In [9]:
score = model.evaluate_generator(gen_test,steps=1024)
score

[0.0818509245857797, 0.973388671875]

In [10]:
model.save('models/model5.h5')